<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4%EC%9B%90%EA%B2%A9)%EC%A6%9D%EB%B6%84G_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 177 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595728 sha256=06db7bce28c38fe6d76e360442a93f8e96125d3c206c6a7931fb240983d891e9
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.2 MB/s 
     |████████████████████████████████| 769 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 82.0 MB/s 
     |████████████████████████████████| 3.0 MB 18.4 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-gkg79e_2
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   산업분류자동화	'Colab Notebooks'  'My Drive'


In [4]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/inc/노이) df_G_inc_again.csv')

## 데이터 전처리

In [5]:
df['digit_3'].value_counts()

464.0    36906
474.0    36453
463.0    35746
468.0    34700
478.0    30566
465.0    29304
473.0    27936
479.0    27654
471.0    26189
475.0    26062
462.0    25850
472.0    25668
466.0    24462
467.0    24192
453.0    23550
452.0    23480
461.0    23280
477.0    22885
451.0    22680
476.0    21564
Name: digit_3, dtype: int64

In [6]:
df.loc[(df['digit_3'] == 451), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 452), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 453), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 461), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 462), 'digit_3'] = 4  
df.loc[(df['digit_3'] == 463), 'digit_3'] = 5  
df.loc[(df['digit_3'] == 464), 'digit_3'] = 6
df.loc[(df['digit_3'] == 465), 'digit_3'] = 7  
df.loc[(df['digit_3'] == 466), 'digit_3'] = 8  
df.loc[(df['digit_3'] == 467), 'digit_3'] = 9  
df.loc[(df['digit_3'] == 468), 'digit_3'] = 10  
df.loc[(df['digit_3'] == 471), 'digit_3'] = 11  
df.loc[(df['digit_3'] == 472), 'digit_3'] = 12  
df.loc[(df['digit_3'] == 473), 'digit_3'] = 13
df.loc[(df['digit_3'] == 474), 'digit_3'] = 14  
df.loc[(df['digit_3'] == 475), 'digit_3'] = 15  
df.loc[(df['digit_3'] == 476), 'digit_3'] = 16  
df.loc[(df['digit_3'] == 477), 'digit_3'] = 17  
df.loc[(df['digit_3'] == 478), 'digit_3'] = 18  
df.loc[(df['digit_3'] == 479), 'digit_3'] = 19

In [7]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['가게에서 고객에게 잡화판매', '11.0'], ['슈퍼마켓  ', '11.0'], ['일정한 시설에서(165㎡미만) 일반소비자에게 음식료품 위주 소매', '11.0'], ['슈퍼에서 일반소비자를 대상으로 생필품 잡화 소매', '11.0'], ['편의점 일반인 대상 식표품 판매', '11.0']]


## Train data & test data

In [8]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [11]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [12]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [13]:
data_train[0]

(array([   2, 2609, 7184,  517, 5859, 6168,  666,  333,  290,  517,   54,
         993, 6825, 5330, 6664,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(16, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 9)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [17]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/7723 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 3.0865559577941895 train acc 0.0625
epoch 1 batch id 201 loss 3.0328314304351807 train acc 0.05363805970149254
epoch 1 batch id 401 loss 2.6831507682800293 train acc 0.0780860349127182
epoch 1 batch id 601 loss 1.838204026222229 train acc 0.1708870632279534
epoch 1 batch id 801 loss 1.159807562828064 train acc 0.3010104556803995
epoch 1 batch id 1001 loss 0.7774277925491333 train acc 0.4020510739260739
epoch 1 batch id 1201 loss 0.5942803621292114 train acc 0.4755542256452956
epoch 1 batch id 1401 loss 0.6794598698616028 train acc 0.5293428800856531
epoch 1 batch id 1601 loss 0.3035777807235718 train acc 0.5695073391630231
epoch 1 batch id 1801 loss 0.5989323854446411 train acc 0.6019659217101611
epoch 1 batch id 2001 loss 0.5743073225021362 train acc 0.6281859070464768
epoch 1 batch id 2201 loss 0.48720690608024597 train acc 0.6498750567923671
epoch 1 batch id 2401 loss 0.4315280020236969 train acc 0.6682567159516868
epoch 1 batch id 2601 loss 0.314726561307907

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9299512514551804


  0%|          | 0/7723 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.05981075018644333 train acc 0.96875
epoch 5 batch id 201 loss 0.2641848623752594 train acc 0.9428638059701493
epoch 5 batch id 401 loss 0.24944402277469635 train acc 0.9419809850374065
epoch 5 batch id 601 loss 0.1627686470746994 train acc 0.9419717138103162
epoch 5 batch id 801 loss 0.11107909679412842 train acc 0.9420060861423221
epoch 5 batch id 1001 loss 0.19095075130462646 train acc 0.9419642857142857
epoch 5 batch id 1201 loss 0.08269134163856506 train acc 0.94244379683597
epoch 5 batch id 1401 loss 0.2118663638830185 train acc 0.9427975553176302
epoch 5 batch id 1601 loss 0.060414232313632965 train acc 0.9426530293566521
epoch 5 batch id 1801 loss 0.10033049434423447 train acc 0.9431739311493614
epoch 5 batch id 2001 loss 0.25569504499435425 train acc 0.9429035482258871
epoch 5 batch id 2201 loss 0.23195099830627441 train acc 0.9427036006360745
epoch 5 batch id 2401 loss 0.10828857868909836 train acc 0.9426866409829238
epoch 5 batch id 2601 loss 0.07719

  0%|          | 0/859 [00:00<?, ?it/s]

epoch 5 test acc 0.932352299185099


In [18]:
#학습 모델 저장
torch.save(model, 'G_inc_again_so_model.pt')

In [21]:
model_save_name = 'G_inc_again_so_model.pt' #드라이브에 저장(잘불러와지는지는 봐야함..... 나는 안불러와져서 컴퓨터에 다운받은거)
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [19]:
print('Done!')

Done!


In [20]:
1+2

3